<a href="https://colab.research.google.com/github/gedalyas/PDFTOEXCEL/blob/main/TransformandoPDFemExcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PDF PARA EXCEL NOVA CASA

In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install PyPDF2

import re
import pandas as pd
from PyPDF2 import PdfReader

# Abrir PDF
pdf_path = "/content/pedido-nova-casa-210825.pdf" #NOME DO PDF
reader = PdfReader(pdf_path)

dados_extraidos = []

# Adicionar uma contagem para exibir as linhas específicas
line_number = 0

for page in reader.pages:
    text = page.extract_text()
    if text:
        lines = text.split('\n')
        for line in lines:
            line_number += 1


            match = re.match(
                r"(\d+)\s+(\d+)\s+(.+?)\s+(UN|LA|PT|CX|JG|C/001)\s+(\d+)\s+\$(\d+,\d+|\d+\.\d+)\s+(\d+)\$(\d+,\d+|\d+\.\d+)(.*?)\s+([A-Za-z0-9/]+)",  # Alteração na coluna "Embalagem"
                line
            )
            if match:
                # Formatar com vírgula no padrão brasileiro
                preco_unit = match.group(6).replace(".", ",").strip()
                preco_total = match.group(8).replace(".", ",").strip()

                # Arredondar os valores para duas casas decimais
                preco_unit = f"{float(preco_unit.replace(',', '.')):.2f}".replace('.', ',')
                preco_total = f"{float(preco_total.replace(',', '.')):.2f}".replace('.', ',')

                dados_extraidos.append({
                    "Item": match.group(1),
                    "Código": match.group(2),
                    "Descrição": match.group(3).strip(),
                    "Unidade": match.group(4),
                    "Quantidade": match.group(5),
                    "Preço Unitário": preco_unit,
                    "Código de Barras": match.group(7),
                    "Preço Total": preco_total,  # Limitar a 2 casas decimais
                    "Embalagem": match.group(9).strip(),
                })

# Criar DataFrame
df = pd.DataFrame(dados_extraidos)

# Salvar em Excel
excel_path = "/content/Pedido_igor_convertido.xlsx" #NOME DO ARQUIVO DE SAIDA
df.to_excel(excel_path, index=False)

# Download no Colab
from google.colab import files
files.download(excel_path)


Saving pedido-nova-casa-210825.pdf to pedido-nova-casa-210825.pdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'

In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install PyPDF2

import re
import pandas as pd
from PyPDF2 import PdfReader

pdf_path = "/content/COMERCIAL TEODORO LIMA LTDA.pdf"
reader = PdfReader(pdf_path)

dados = []

for page in reader.pages:
    text = page.extract_text()
    if text:
        lines = text.split('\n')
        for line in lines:
            match = re.match(
                r"^(\d+)\s+(\d+)\s+([A-Z0-9./-]+)\s+(.+?)\s+(\d+)\s+(\d{8})\s+(IMP|NAC)\s+([\d,]+)%\s+([\d,]+)%\s+(\d+)\s+([\d,]+)\s+([\d,]+)%\s+([\d,]+)%\s+([\d,]+)\s+([\d,]+)\s+([\d,]+)%\s+([\d,]+)\s+([\d,]+)",
                line
            )
            if match:
                dados.append({
                    "Item": match.group(1),
                    "Qtde": match.group(2),
                    "Código": match.group(3),
                    "Descrição": match.group(4).strip(),
                    "Embalagem": match.group(5),
                    "NCM": match.group(6),
                    "Origem": match.group(7),
                    "% ICMS-ST": match.group(8),
                    "% ICMS DCAT": match.group(9),
                    "CFOP": match.group(10),
                    "Preço Bruto": match.group(11),
                    "Desc. Total (%)": match.group(12),
                    "Desc. Origem (%)": match.group(13),
                    "Preço Unitário (R$)": match.group(14),
                    "Valor Total (R$)": match.group(15),
                    "% IPI": match.group(16),
                    "Valor IPI (R$)": match.group(17),
                    "Valor ICMS-ST (R$)": match.group(18),
                })

# Criação do DataFrame
df = pd.DataFrame(dados)

# Exportar para Excel
excel_path = "/content/Pedido_Teodoro_convertido.xlsx"
df.to_excel(excel_path, index=False)

# Download no Colab
from google.colab import files
files.download(excel_path)


#CISER

In [ ]:
# 1) Instalar dependências
!pip -q install pdfplumber openpyxl

# 2) Upload e seleção automática do PDF
from google.colab import files
import os, pdfplumber, pandas as pd

uploaded = files.upload()  # envie seu(s) PDF(s)

# pega o primeiro PDF enviado (ou aquele cujo nome você quiser)
pdf_names = [n for n in uploaded.keys() if n.lower().endswith('.pdf')]
if not pdf_names:
    raise ValueError("Nenhum PDF enviado. Envie um arquivo .pdf.")

pdf_filename = pdf_names[0]                  # ou troque para o nome exato, se quiser
pdf_path = f"/content/{pdf_filename}"
print(f"Usando arquivo: {pdf_filename}")

# 3) Extração das tabelas
tabela_final = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tabelas = page.extract_tables() or []
        for tabela in tabelas:
            for linha in (tabela or []):
                if not linha:
                    continue
                first = ("" if linha[0] is None else str(linha[0]).strip())
                # ignora cabeçalho
                if first.lower() == "item" or first == "":
                    continue
                tabela_final.append(linha)

# 4) Normalização e DataFrame
colunas = [
    "Item", "Div", "Produto", "Produto Cliente", "Descrição", "NCM", "UMV", "Emb.",
    "Qtd.", "Ordem Compra", "Preço S/ IPI", "Preço C/ IPI ST", "IPI (%)",
    "ST (R$)", "ICMS", "Total S/ IPI", "Total C/ IPI", "Data Remessa"
]

linhas_norm = []
for linha in tabela_final:
    # troca None por "", tira espaços e garante comprimento certo
    row = ["" if v is None else str(v).strip() for v in linha]
    if len(row) < len(colunas):
        row += [""] * (len(colunas) - len(row))
    elif len(row) > len(colunas):
        row = row[:len(colunas)]
    linhas_norm.append(row)

df = pd.DataFrame(linhas_norm, columns=colunas)

# mantém apenas linhas onde "Item" é numérico
df = df[df["Item"].astype(str).str.strip().str.isnumeric()].reset_index(drop=True)

# 5) Salva e baixa Excel
excel_path = f"/content/{os.path.splitext(pdf_filename)[0]}_tabelas.xlsx"
df.to_excel(excel_path, index=False)

from google.colab import files as colab_files
colab_files.download(excel_path)

# opcional: ver prévia
df.head()


Saving 0021388082.PDF to 0021388082 (1).PDF
Usando arquivo: 0021388082 (1).PDF


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Item,Div,Produto,Produto Cliente,Descrição,NCM,UMV,Emb.,Qtd.,Ordem Compra,Preço S/ IPI,Preço C/ IPI ST,IPI (%),ST (R$),ICMS,Total S/ IPI,Total C/ IPI,Data Remessa
0,10,1,18641104,,PA CL SI AC 12.9 MA 10X30 RI ET,7318.15.00,CE Centos,"1,00",1,,"45,3700","56,8800","6,50","8,56",12,"45,37","56,88",03.09.2025
1,20,1,18746604,,PA CL SI AC 12.9 MA 12X55 RP ET,7318.15.00,CE Centos,"1,00",1,,"97,5000","122,2300","6,50","18,39",12,"97,50","122,23",03.09.2025
2,30,1,16025504,,PA SX AC MA 8.8 6x16 RI ET,7318.15.00,CE Centos,"5,00",15,,"9,9200","12,4353","6,50","28,06",12,"148,80","186,53",03.09.2025
3,40,1,16025704,,PA SX AC MA 8.8 6x20 RI ET,7318.15.00,CE Centos,"5,00",15,,"10,6500","13,3507","6,50","30,13",12,"159,75","200,26",03.09.2025
4,50,1,16025904,,PA SX AC MA 8.8 6x25 RI ET,7318.15.00,CE Centos,"2,00",16,,"12,6500","15,8581","6,50","38,17",12,"202,40","253,73",03.09.2025


#Metalbo


In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install pdfplumber openpyxl

import pdfplumber
import pandas as pd

pdf_path = "/content/metalbo 1.pdf"
dados = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        texto = page.extract_text()
        linhas = texto.split('\n')

        for linha in linhas:
            partes = linha.strip().split()

            # Verifica se a linha tem pelo menos 8 partes (para garantir estrutura)
            if len(partes) >= 8:
                try:
                    seq = partes[0]
                    codigo = partes[1]
                    qtd_ct = partes[-4]
                    preco_unit = partes[-3]
                    aliq_icms = partes[-2]
                    total = partes[-1]
                    descricao = " ".join(partes[2:-4])  # Tudo entre o código e a quantidade

                    # MODIFICAÇÃO 1: Verifica se o ICMS é "4,00" ou "12,00" (com vírgula)
                    if len(codigo) == 8 and aliq_icms in ["4,00", "12,00"]:

                        # MODIFICAÇÃO 2: Adiciona os dados mantendo a vírgula original
                        dados.append([
                            seq,
                            codigo,
                            descricao,
                            qtd_ct,
                            preco_unit,
                            aliq_icms,
                            total
                        ])
                except:
                    continue

# Monta o DataFrame
colunas = ["Seq", "Codigo", "Descrição", "Qtd Ct", "R$", "Aliq.icms", "R$ Total"]
df = pd.DataFrame(dados, columns=colunas)

# Salva como Excel
excel_path = "/content/pedido_metalbo_certo.xlsx"
df.to_excel(excel_path, index=False)
files.download(excel_path)

#BELENUS

In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install pdfplumber openpyxl

import pdfplumber
import pandas as pd

pdf_path = "/content/CotBelComple.pdf"
tabela_final = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tabelas = page.extract_tables()
        for tabela in tabelas:
            for linha in tabela:
                if linha and linha[0].strip() not in ["Item", "Código"]:
                      tabela_final.append(linha)

colunas = [
    "Código", "Produto", "Un.", "Qtde.", "Emb.", "Preço", "Subtotal", "CST",
    "%ICM", "%IPI", "%ST", "Tot + IPI + ST", "Entrega"
]
for i, linha in enumerate(tabela_final):
    if len(linha) < len(colunas):
        linha.extend([""] * (len(colunas) - len(linha)))
    elif len(linha) > len(colunas):
        tabela_final[i] = linha[:len(colunas)]

df = pd.DataFrame(tabela_final, columns=colunas)
excel_path = "/content/CotBelenusCompleQuebrado.xlsx"
df.to_excel(excel_path, index=False)
files.download(excel_path)

Saving CotBelComple.pdf to CotBelComple.pdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 342.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 941.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.1 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#MULTI FIX

In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install pdfplumber pandas openpyxl

import pdfplumber
import pandas as pd
import re

# Caminho do PDF
pdf_path = list(uploaded.keys())[0]

# Lista para armazenar as linhas extraídas
tabela_final = []

# Novo regex ajustado com 14 colunas: incluindo Item e separando Código da Descrição
padrao_linha = re.compile(
    r'^(\d+)\s+(\d{4,})\s+(.+?)\s+(CT|UN)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)$'
)

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        texto = page.extract_text()
        for linha in texto.split('\n'):
            match = padrao_linha.match(linha.strip())
            if match:
                tabela_final.append(list(match.groups()))

# Nome das colunas no mesmo padrão do PDF
colunas = [
    "Item", "Código", "Descrição do Produto", "UN", "Qtde.",
    "Valor Unit.", "Valor Total", "Aliq. ICMS", "Valor ICMS",
    "Aliq. IPI", "Valor IPI", "ICMS-ST", "Unit + Impostos", "Total c/ Impostos"
]

# Criar e salvar DataFrame
df = pd.DataFrame(tabela_final, columns=colunas)
excel_path = "/content/pedido_multifix_corrigido.xlsx"
df.to_excel(excel_path, index=False)

# Fazer download do Excel gerado
files.download(excel_path)


Saving MULTI_FIX 2.pdf to MULTI_FIX 2 (3).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#REX

In [ ]:
!pip install pdfplumber pandas openpyxl
import pdfplumber
import pandas as pd
import re
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

colunas = [
    "Seq.", "Produto", "Cod. Cli.", "Descrição do Produto", "OC", "Seq. Item", "Un",
    "Quantid. Pedido", "Preço Unit.", "ICMS ST", "% ICMS", "Total", "% IPI",
    "Total C/ Imposto", "Peso", "Embarq.", "Est."
]

linhas_final = []

with pdfplumber.open(pdf_path) as pdf:
    linhas = []
    for pagina in pdf.pages:
        texto = pagina.extract_text()
        if texto:
            linhas.extend(texto.split('\n'))

i = 0
while i < len(linhas):
    linha = linhas[i].strip()

    if len(re.findall(r'\b[A-Z]{3,}\b', linha)) > 5:
        i += 1
        continue

    match = re.match(r'^(\d+)\s+(\d+)\s+(.+?)\s+([A-Z]{2})\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+([\d,.]+)\s+(\d{2}/\d{2}/\d{4})', linha)

    if match:
        seq = match.group(1)
        produto = match.group(2)
        desc_ini = match.group(3).strip()
        un = match.group(4)
        qtd = match.group(5)
        preco = match.group(6)
        icms_st = match.group(7)
        perc_icms = match.group(8)
        total = match.group(9)
        perc_ipi = match.group(10)
        total_com = match.group(11)
        peso = match.group(12)
        embarque = match.group(13)

        descricao = desc_ini

        if i + 1 < len(linhas):
            prox1 = linhas[i + 1].strip()
            if (not re.search(r'\d{2}/\d{2}/\d{4}', prox1)
                and len(prox1) > 3
                and not re.search(r'Valor Produtos|Valor Frete|Peso Total', prox1, re.IGNORECASE)):
                descricao += " " + prox1
                i += 1

        if i + 1 < len(linhas):
            prox2 = linhas[i + 1].strip()
            if (not re.search(r'\d{2}/\d{2}/\d{4}', prox2)
                and len(prox2) > 3
                and not re.search(r'Valor Produtos|Valor Frete|Peso Total', prox2, re.IGNORECASE)):
                descricao += " " + prox2
                i += 1

        linha_dict = {
            "Seq.": seq,
            "Produto": produto,
            "Cod. Cli.": "",
            "Descrição do Produto": descricao,
            "OC": "",
            "Seq. Item": "",
            "Un": un,
            "Quantid. Pedido": qtd,
            "Preço Unit.": preco,
            "ICMS ST": icms_st,
            "% ICMS": perc_icms,
            "Total": total,
            "% IPI": perc_ipi,
            "Total C/ Imposto": total_com,
            "Peso": peso,
            "Embarq.": embarque,
            "Est.": ""
        }

        linhas_final.append(linha_dict)
        i += 1
    else:
        i += 1

df = pd.DataFrame(linhas_final, columns=colunas)

excel_path = "/content/CotRexFinal.xlsx"
df.to_excel(excel_path, index=False)
files.download(excel_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.2 MB/s eta 0:00:00


Saving cot rex 1 rest.pdf to cot rex 1 rest.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# NEW FIX

In [ ]:
!pip install pdfplumber pandas openpyxl
import pdfplumber
import pandas as pd
import re
from google.colab import files

# Upload do PDF
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Colunas com unidade separada
colunas = [
    "Código", "Descrição do Produto", "Unidade", "Quantidade",
    "Unitário Bruto", "% Desc.", "Unitário Líquido",
    "Total", "% IPI", "% ICMS", "ICMS ST", "Data Entrega"
]

linhas_final = []

# Ler PDF
with pdfplumber.open(pdf_path) as pdf:
    linhas = []
    for pagina in pdf.pages:
        texto = pagina.extract_text()
        if texto:
            linhas.extend(texto.split('\n'))

# Regex com grupo da unidade separado
padrao = re.compile(
    r'^(\d{7,8})\s+(.+?)\s+([A-Z]{2,5})\s+([\d.,]+)\s+([\d.,]+)\s+([\d.,]+)\s+([\d.,]+)\s+([\d.,]+)\s+([\d.,]+)\s+([\d.,]+)\s+([\d.,]+)\s+(\d{2}/\d{2}/\d{4})$'
)

# Processar as linhas
for linha in linhas:
    linha = linha.strip()
    match = padrao.match(linha)
    if match:
        linha_dict = {
            "Código": match.group(1),
            "Descrição do Produto": match.group(2),
            "Unidade": match.group(3),
            "Quantidade": match.group(4),
            "Unitário Bruto": match.group(5),
            "% Desc.": match.group(6),
            "Unitário Líquido": match.group(7),
            "Total": match.group(8),
            "% IPI": match.group(9),
            "% ICMS": match.group(10),
            "ICMS ST": match.group(11),
            "Data Entrega": match.group(12)
        }
        linhas_final.append(linha_dict)

# Gerar Excel
df = pd.DataFrame(linhas_final, columns=colunas)
excel_path = "/content/Pedido_NewFix_Com_Unidade.xlsx"
df.to_excel(excel_path, index=False)
files.download(excel_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.6 MB/s eta 0:00:00


Saving cot chipboard.pdf to cot chipboard.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#JOMARCA

#ANCORA

In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install pdfplumber pandas openpyxl

import pdfplumber
import pandas as pd

# Caminho do arquivo PDF enviado
pdf_path = list(uploaded.keys())[0]

# Lista para armazenar os dados da tabela
tabela_final = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tabelas = page.extract_tables()
        for tabela in tabelas:
            for linha in tabela:
                # Evitar cabeçalhos
                if linha and linha[0] and "Item" not in linha[0]:
                    tabela_final.append(linha)

# Definindo as 12 colunas corretas
colunas = [
    "Item", "Produto", "Descrição", "UM", "Qtde.",
    "Valor Unit. (R$)", "Valor Total (R$)", "%IPI", "%ICMS",
    "S.T. (R$)", "Total c/ Imp. (R$)", "Prev. Embarq."
]

# Ajustar linhas com colunas a mais ou a menos
for i, linha in enumerate(tabela_final):
    if len(linha) < len(colunas):
        linha.extend([""] * (len(colunas) - len(linha)))
    elif len(linha) > len(colunas):
        tabela_final[i] = linha[:len(colunas)]

# Criar DataFrame e salvar no Excel
df = pd.DataFrame(tabela_final, columns=colunas)
excel_path = "/content/pedido_ancora_completo.xlsx"
df.to_excel(excel_path, index=False)

# Download do arquivo gerado
files.download(excel_path)


Saving pedido ancora.pdf to pedido ancora (1).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#WALSIWA

In [ ]:
from google.colab import files
uploaded = files.upload()

!pip install pdfplumber pandas openpyxl

import pdfplumber
import pandas as pd
import re

pdf_path = list(uploaded.keys())[0]
dados = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        texto = page.extract_text()
        linhas = texto.split('\n')

        for linha in linhas:
            linha = linha.strip()

            # Agora aceita "Item" com número ou letra, tipo: 01, 99, 9A, 10A etc.
            if re.match(r"^[\dA-Z]{2,}\s+\d{6}", linha) and "PC" in linha:
                # Junta "R$" com o número que vem depois
                linha = re.sub(r'R\$\s*([\d.,]+)', r'R$\1', linha)
                partes = linha.split()

                try:
                    idx_pc = partes.index("PC")

                    item = partes[0]
                    codigo = partes[1]
                    descricao = ' '.join(partes[2:idx_pc])
                    un = "PC"
                    quant = partes[idx_pc + 1]
                    prc_unit = partes[idx_pc + 2].replace("R$", "")
                    total = partes[idx_pc + 3].replace("R$", "")
                    ipi = partes[idx_pc + 4]
                    icms = partes[idx_pc + 5]
                    subst = partes[idx_pc + 6]
                    peso = partes[idx_pc + 7]
                    validade = partes[idx_pc + 8] if len(partes) > idx_pc + 8 else "-"

                    dados.append([
                        item, codigo, descricao, un, quant,
                        prc_unit, total, ipi, icms,
                        subst, peso, validade
                    ])
                except Exception as e:
                    print("❌ Erro ao processar linha:")
                    print(linha)
                    print(e)

# Definir colunas
colunas = [
    "Item", "Código", "Descrição", "UN", "Quant",
    "Prç. Unit.", "Total", "IPI (%)", "ICM (%)",
    "Subst. Trib.", "Peso total", "Dt. Validade"
]

df = pd.DataFrame(dados, columns=colunas)

# Exporta para Excel
excel_path = "/content/cotacao_walsywa.xlsx"
df.to_excel(excel_path, index=False)

# Download automático
files.download(excel_path)


Saving cotação walsiwa.pdf to cotação walsiwa (7).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#FEY

In [ ]:
!pip install pdfplumber openpyxl

import pdfplumber
import pandas as pd
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

linhas_corrigidas = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if not row:
                    continue
                primeira_celula = row[0]
                if (
                    not primeira_celula or
                    str(primeira_celula).strip().upper() in ["SEQ", "ITENS"]
                ):
                    continue
                linha = [cell.strip() if cell else "" for cell in row]
                if len(linha) == 14:
                    linha[11] = linha[11] + " " + linha[12]
                    del linha[12]
                if len(linha) < 13:
                    linha += [""] * (13 - len(linha))
                elif len(linha) > 13:
                    linha = linha[:13]

                linhas_corrigidas.append(linha)

df = pd.DataFrame(linhas_corrigidas)
df = df.drop(index=[0, 1, 2, 186, 187, 188], errors="ignore").reset_index(drop=True)
excel_path = "/content/fey_2_corrigido_final.xlsx"
df.to_excel(excel_path, index=False, header=False)

files.download(excel_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.9 MB/s eta 0:00:00


Saving fey 1.pdf to fey 1.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#MTX

In [ ]:
# 1. Instalação das bibliotecas necessárias
!pip install pdfplumber pandas openpyxl

# 2. Importação das bibliotecas e upload do arquivo
import pdfplumber
import pandas as pd
import re
from google.colab import files

print("Por favor, faça o upload do seu arquivo PDF.")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# 3. Função para limpar e converter valores monetários
def clean_price(price_str):
    """Remove 'R$', formata e converte a string de preço para float."""
    if not price_str or not isinstance(price_str, str):
        return None
    price_str = price_str.replace('R$', '').strip()
    # Remove o separador de milhares e substitui a vírgula decimal por ponto
    price_str = price_str.replace('.', '').replace(',', '.')
    # Lida com possíveis erros de OCR com múltiplos pontos decimais
    if price_str.count('.') > 1:
        parts = price_str.split('.')
        price_str = parts[0] + '.' + ''.join(parts[1:])
    try:
        return float(price_str)
    except (ValueError, TypeError):
        return None

# 4. Processamento do PDF
print(f"Processando o arquivo: {pdf_path}")

# Definição das colunas do DataFrame
colunas = [
    "#", "Código", "Produto", "NCM", "Qtde.", "Preço Tabela", "Desc.",
    "Preço Líquido", "Alíq. IPI", "Aliq. ICMS-ST", "Preço Liq. c/ Impostos",
    "Subtotal (sem IPI/ST)", "Subtotal IPI", "Subtotal ICMS-ST", "Subtotal"
]
max_cols = len(colunas)

data_rows = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        print(f"Extraindo dados da página {i+1}...")
        table = page.extract_table()

        if table:
            for row in table:
                cleaned_row = [str(cell).strip().replace('\n', ' ') if cell is not None else '' for cell in row]

                if cleaned_row and re.match(r'^\d+$', cleaned_row[0]):
                    # *** CORREÇÃO APLICADA AQUI ***
                    # Garante que a linha tenha no máximo max_cols colunas antes de padronizar
                    processed_row = cleaned_row[:max_cols]
                    padded_row = processed_row + [''] * (max_cols - len(processed_row))
                    data_rows.append(padded_row)
                elif data_rows and cleaned_row and not cleaned_row[0] and len(cleaned_row) > 2 and cleaned_row[2]:
                    data_rows[-1][2] += ' ' + cleaned_row[2]

# 5. Criação e limpeza do DataFrame
df = pd.DataFrame(data_rows, columns=colunas)

# Limpa as colunas de preço
price_columns = ["Preço Tabela", "Preço Líquido", "Preço Liq. c/ Impostos",
                 "Subtotal (sem IPI/ST)", "Subtotal IPI", "Subtotal ICMS-ST", "Subtotal"]
for col in price_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_price)

# Limpa as colunas de porcentagem
percent_columns = ["Desc.", "Alíq. IPI", "Aliq. ICMS-ST"]
for col in percent_columns:
     if col in df.columns:
        df[col] = df[col].str.replace('%', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove linhas que possam ter sido criadas incorretamente
df.dropna(subset=['Código'], inplace=True)
df = df[df['Código'] != '']
df = df[df['Código'] != 'Código']

# 6. Exportação para Excel e Download
excel_path = "promocao_mtx_extraido.xlsx"
df.to_excel(excel_path, index=False)

print("\nProcessamento concluído com sucesso!")
print(f"O arquivo Excel foi salvo como: {excel_path}")

files.download(excel_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.2 MB/s eta 0:00:00
Por favor, faça o upload do seu arquivo PDF.


Saving retorno pedido mtx.pdf to retorno pedido mtx.pdf
Processando o arquivo: retorno pedido mtx.pdf
Extraindo dados da página 1...
Extraindo dados da página 2...
Extraindo dados da página 3...
Extraindo dados da página 4...
Extraindo dados da página 5...

Processamento concluído com sucesso!
O arquivo Excel foi salvo como: promocao_mtx_extraido.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#LOTUS

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Extrai itens do PDF de pedido (layout Lotus) e exporta para Excel.

- Robusto para rodar no Google Colab (auto-instala dependências).
- Ignora argumentos extras que o Colab/Jupyter passam (-f ...).

Como usar no Colab (2 jeitos):
A) Mais simples (arquivo .py):
   1) from google.colab import files; files.upload()  # suba o PDF
   2) !python pdf_pedido_lotus_to_excel.py --pdf "SEU_ARQUIVO.pdf" --out "itens.xlsx"

B) Colando este código numa célula:
   - Execute a célula; se você não passar --pdf, ele abre o upload automaticamente.
"""

import sys, subprocess, pkgutil, re, argparse
from typing import List, Dict, Optional

# ---------- Auto-instalação ----------
def ensure(package: str):
    if pkgutil.find_loader(package) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for p in ["pdfplumber", "pandas", "openpyxl"]:
    ensure(p)

import pdfplumber  # type: ignore
import pandas as pd  # type: ignore

# ---------- Regex e auxiliares ----------
MONEY_RE = r'\d{1,3}(?:\.\d{3})*,\d{2}'  # 1.234,56 ou 12,34
HEADER_RE = re.compile(
    r'\bS[EÉ]Q\b.*C[OÓ]DIGO\b.*Q[Tt]\b.*DESCRI[ÇC][ÃA]O\b',
    flags=re.IGNORECASE
)
SKIP_HEADERS = [
    re.compile(r'\bORDEM\s+REF\s+QT\b', re.IGNORECASE),
    re.compile(r'PRE[ÇC]OS.*\bST\b', re.IGNORECASE),
]

def clean_money_tokens(s: str) -> List[str]:
    """Extrai valores monetários em formato brasileiro (ex.: 1.234,56)."""
    s = re.sub(r'R\$\s*', '', s)  # remove R$ e espaços
    s = s.replace('–', '-').replace('—', '-')
    return re.findall(MONEY_RE, s)

def br_to_float(txt: Optional[str]) -> Optional[float]:
    if not txt:
        return None
    return float(txt.replace('.', '').replace(',', '.'))

# ---------- Parser principal ----------
def parse_pdf(pdf_path: str) -> pd.DataFrame:
    rows: List[Dict] = []
    in_table = False
    last_row_idx: Optional[int] = None

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            for raw in text.splitlines():
                line = raw.strip()
                if not line:
                    continue

                # Detecta início da tabela pelo cabeçalho
                if HEADER_RE.search(line):
                    in_table = True
                    continue

                # Ignora cabeçalhos/seções recorrentes
                if any(p.search(line) for p in SKIP_HEADERS):
                    continue

                if not in_table:
                    continue

                # Padrão de item: SEQ  CÓDIGO  QT  DESCRIÇÃO ... [preços]
                m = re.match(r'^(\d+)\s+([A-Z0-9\.\-]+)\s+(\d+)\s+(.*)$', line, flags=re.IGNORECASE)
                if m:
                    seq = int(m.group(1))
                    codigo = m.group(2)
                    try:
                        qt = int(m.group(3))
                    except ValueError:
                        continue
                    tail = m.group(4)

                    # Valores monetários (2 a 4)
                    money = clean_money_tokens(tail)

                    # Descrição até o primeiro valor monetário
                    mpos = re.search(MONEY_RE, tail)
                    if mpos:
                        descricao = tail[:mpos.start()].strip(" -")
                    else:
                        descricao = tail.strip()

                    uni_s_st   = money[0] if len(money) > 0 else None
                    total_s_st = money[1] if len(money) > 1 else None
                    uni_c_st   = money[2] if len(money) > 2 else None
                    total_c_st = money[3] if len(money) > 3 else None

                    rows.append({
                        "Seq": seq,
                        "Código": codigo,
                        "Quantidade": qt,
                        "Descrição": descricao,
                        "UN S/ ST": br_to_float(uni_s_st),
                        "TOTAL S/ ST": br_to_float(total_s_st),
                        "UN C/ ST": br_to_float(uni_c_st),
                        "TOTAL C/ ST": br_to_float(total_c_st),
                    })
                    last_row_idx = len(rows) - 1
                    continue

                # Possível continuação de descrição
                if last_row_idx is not None:
                    if HEADER_RE.search(line) or any(p.search(line) for p in SKIP_HEADERS):
                        continue
                    if not re.search(MONEY_RE, line):
                        if rows[last_row_idx]["Descrição"]:
                            rows[last_row_idx]["Descrição"] += " " + line
                        else:
                            rows[last_row_idx]["Descrição"] = line
                        continue

    df = pd.DataFrame(rows)
    if not df.empty:
        df = df.sort_values(by=["Seq", "Código"], kind="stable").reset_index(drop=True)
    return df

# ---------- CLI / Colab ----------
def main():
    # Detecta Colab (para download e upload automáticos)
    try:
        import google.colab  # type: ignore
        running_in_colab = True
    except Exception:
        running_in_colab = False

    parser = argparse.ArgumentParser(description="Extrai itens do PDF de pedido (layout Lotus) para Excel.")
    parser.add_argument("--pdf", help="Caminho do arquivo PDF de entrada.")
    parser.add_argument("--out", default="itens_pedido_lotus.xlsx", help="Arquivo .xlsx de saída (default: itens_pedido_lotus.xlsx).")

    # <<< Importante >>>: ignora argumentos desconhecidos (ex.: '-f' do Colab/Jupyter)
    args, _unknown = parser.parse_known_args()

    pdf_path = args.pdf

    # Se estiver no Colab e não foi informado --pdf, abre janela de upload
    if running_in_colab and not pdf_path:
        try:
            from google.colab import files  # type: ignore
            print("Nenhum --pdf informado. Abrindo janela de upload...")
            uploaded = files.upload()
            if not uploaded:
                print("Nenhum arquivo enviado. Encerrando.")
                sys.exit(1)
            pdf_path = list(uploaded.keys())[0]
        except Exception as e:
            print(f"Falha no upload do Colab: {e}")
            sys.exit(1)

    if not pdf_path:
        print("Erro: informe --pdf CAMINHO_DO_PDF")
        sys.exit(2)

    print(f"Lendo: {pdf_path}")
    df = parse_pdf(pdf_path)

    if df.empty:
        print("Aviso: nenhum item encontrado. Verifique o layout do PDF.")
    else:
        print(f"Itens extraídos: {len(df)}")

    out_path = args.out
    df.to_excel(out_path, index=False)
    print(f"Arquivo Excel gerado em: {out_path}")

    if running_in_colab:
        try:
            from google.colab import files  # type: ignore
            files.download(out_path)
        except Exception:
            pass

if __name__ == "__main__":
    main()


Nenhum --pdf informado. Abrindo janela de upload...


Saving pedido lotus.pdf to pedido lotus.pdf
Lendo: pedido lotus.pdf
Itens extraídos: 59
Arquivo Excel gerado em: itens_pedido_lotus.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>